# Building a Context-Aware Content Recommendation System Using BERT Embeddings | News Articles Recommedations Using Sentence Transformers


# Step 1: Import Required Libraries


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Step 2: Load Your Dataset


In [3]:
# Try reading the file with a different encoding
df = pd.read_csv('/content/Articles.csv', encoding='ISO-8859-1')


In [4]:
# Inspect the data
df

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business
...,...,...,...,...
2687,strong>DUBAI: Dubai International Airport and ...,3/25/2017,Laptop ban hits Dubai for 11m weekend traveller,business
2688,"strong>BEIJING: Former Prime Minister, Shaukat...",3/26/2017,Pak China relations not against any third coun...,business
2689,strong>WASHINGTON: Uber has grounded its fleet...,3/26/2017,Uber grounds self driving cars after accid,business
2690,strong>BEIJING: The New Development Bank plans...,3/27/2017,New Development Bank plans joint investments i...,business


# Step 3: Preprocessing the Text Data


In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

# Download stopwords and punkt (for tokenization)
nltk.download('stopwords')
nltk.download('punkt')


# Initialize the PorterStemmer and stopwords
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Preprocessing function to clean text (removes punctuation, converts to lowercase, removes stopwords, applies stemming)
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize the text
    words = text.split()

    # Remove stopwords and apply stemming
    processed_words = [stemmer.stem(word) for word in words if word not in stop_words]

    # Join the words back into a string
    return " ".join(processed_words)

# Apply preprocessing to the 'Heading' column (or any other text column you want)
df['cleaned_heading'] = df['Heading'].apply(preprocess_text)

df


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Article,Date,Heading,NewsType,cleaned_heading
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business,sindh govt decid cut public transport fare 7pc...
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business,asia stock new year trad
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business,hong kong stock open 066 percent lower
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business,asian stock sink euro near nine year
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business,us oil price slip 50 barr
...,...,...,...,...,...
2687,strong>DUBAI: Dubai International Airport and ...,3/25/2017,Laptop ban hits Dubai for 11m weekend traveller,business,laptop ban hit dubai 11m weekend travel
2688,"strong>BEIJING: Former Prime Minister, Shaukat...",3/26/2017,Pak China relations not against any third coun...,business,pak china relat third countri shaukat aziz
2689,strong>WASHINGTON: Uber has grounded its fleet...,3/26/2017,Uber grounds self driving cars after accid,business,uber ground self drive car accid
2690,strong>BEIJING: The New Development Bank plans...,3/27/2017,New Development Bank plans joint investments i...,business,new develop bank plan joint invest economi alo...


# Step 4: Initialize The Sentence Transformer Model

In [6]:
# Load the pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can use other models if you prefer

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Step 5: Text Embeddings Using BERT


In [7]:
headings = df['cleaned_heading'].tolist()

embeddings = model.encode(headings, convert_to_tensor=True)

In [8]:
headings[0:5]

['sindh govt decid cut public transport fare 7pc kti rej',
 'asia stock new year trad',
 'hong kong stock open 066 percent lower',
 'asian stock sink euro near nine year',
 'us oil price slip 50 barr']

In [9]:
df.shape

(2692, 5)

In [10]:
embeddings[0:5]

tensor([[-0.0313,  0.0338, -0.0093,  ..., -0.0394,  0.0069,  0.0495],
        [-0.0520,  0.0016, -0.0221,  ..., -0.1266, -0.0875,  0.0417],
        [ 0.0940, -0.0362,  0.0582,  ..., -0.1183, -0.0391,  0.0702],
        [-0.0153,  0.0438,  0.0423,  ..., -0.0971, -0.0474,  0.0380],
        [-0.0803, -0.0079,  0.0873,  ..., -0.1084, -0.0170,  0.0278]],
       device='cuda:0')

# Save and Load Embeddings

In [11]:
import pickle

pickle.dump(embeddings, open("embeddings.pkl", "wb"))

embeddings = pickle.load(open("embeddings.pkl",'rb'))

# Step 6: Recommendations


In [12]:
# Function to recommend top N articles based on a search query
def recommend_articles_from_search(query, df, model, embeddings, num_recommendations=5):
    # Step 0: Clean Query
    query = preprocess_text(query)  # Clean the query text (if needed)

    # Step 1: Get the embedding for the search query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Ensure query_embedding and article embeddings are on the CPU
    query_embedding = query_embedding.cpu()
    embeddings_cpu = embeddings.cpu()

    # Step 2: Compute cosine similarity between the query and all article embeddings
    similarities = cosine_similarity(query_embedding.reshape(1, -1), embeddings_cpu)  # Query should be reshaped

    # Step 3: Get the indices of the top N most similar articles
    similarities = similarities.flatten()  # Flatten to 1D
    top_indices = similarities.argsort()[-num_recommendations:][::-1]  # Get top N indices

    # Step 4: Retrieve the corresponding articles and their similarity scores
    recommended_articles = df.iloc[top_indices][['Heading', 'NewsType', 'Article', 'Date']]  # Adjust columns as needed
    return recommended_articles

# Example usage of the recommendation function
query = "Asian markets upswing"
recommended_articles = recommend_articles_from_search(query, df, model, embeddings, num_recommendations=10)

recommended_articles

,Heading,NewsType,Article,Date
119,most asian markets up tokyo at 15 year hig,business,Hong Kong: Japanese shares hit a 15-year high ...,4/22/2015
226,asian markets mostly recover from hefty sell off,business,Hong Kong: Most Asia shares rose Wednesday as ...,7/29/2015
2610,Dollar down Trump takes over Asia markets u,business,strong>HONG KONG: The dollar retreated against...,1/23/2017
220,asia markets mostly down shanghai up a 7th str...,business,Hong Kong: Asian markets mostly fell Friday fo...,7/24/2015
546,Asia stocks edge up to four month high after W...,business,strong>TOKYO: Asian shares edged up to a four-...,3/31/2016
307,asia markets rebound glencore surges after cr,business,Hong Kong: Asian stocks recovered slightly Wed...,9/30/2015
354,asia markets down after paris attacks japan in...,business,Hong Kong: Asian stock markets tumbled and the...,11/16/2015
432,Asia stocks cheered by China trade surpri,business,strong>SYDNEY: Asian shares made their first r...,1/13/2016
264,asian stocks up for second day on us gr,business,Hong Kong: Tokyo shares led Asian stock market...,8/28/2015
92,asian markets follow wall st lower after us d,business,Hong Kong: Asian markets mostly wound back Thu...,3/26/2015


In [14]:

# Example usage of the recommendation function
query = "India News"
recommended_articles = recommend_articles_from_search(query, df, model, embeddings, num_recommendations=10)

recommended_articles

,Heading,NewsType,Article,Date
2334,Rangana Herath spins Sri Lanka out of troub,sports,strong>PHALLEKELE: Sri Lanka spinner Rangana H...,7/27/2016
1332,India to have an edge against Sri L,sports,"DHAKA: On the face of it, the India v Sri Lan...",3/1/2016
2307,India push wilting Windies to verge of heavy def,sports,"strong>North Sound, Antigua and Barbuda: West ...",7/24/2016
2365,Holder makes breakthrough but India in contr,sports,"KINGSTON, Jamaica: West Indies captain Jason H...",8/1/2016
1996,India shortlist candidates head coach r,sports,strong>NEW DELHI: A panel including former cap...,6/16/2016
1530,West Indies knock India out of Womens World T20,sports,MOHALI: Not even the support of a festive home...,3/27/2016
1677,India announce Windies tour to revive ti,sports,strong>NEW DELHI: India will tour West Indies ...,4/30/2016
2557,Chinese media advises India to ease tensions w...,business,strong>BEIJING: Top Chinese daily Global Times...,12/23/2016
2352,India put Windies under pressure despite Black...,sports,"KINGSTON, Jamaica: Jermaine Blackwood smashed ...",7/30/2016
655,Apple boss Cook meets Modi plot India strategy,business,strong>NEW DELHI/MUMBAI: Apple Inc chief Tim ...,5/21/2016
